In [ ]:
import os

os.chdir("..")

In [ ]:
import cv2
import numpy as np
import matplotlib.pyplot as plt
import urllib.request as urlreq

import scipy.io as scio
from face_frontalization import frontalize
from face_frontalization import camera_calibration as calib

%load_ext autoreload
%autoreload 2

In [ ]:
random_seed = 8
face_image_target_size = (64, 64)

base_folder = "demo/dataset-norm"
haarcascade = "model_checkpoints/haarcascade.xml"
haarcascade_url = "https://raw.githubusercontent.com/opencv/opencv/master/data/haarcascades/haarcascade_frontalface_alt2.xml"
LBFmodel = "model_checkpoints/lbfmodel.yaml"
LBFmodel_url = "https://github.com/kurnianggoro/GSOC2017/raw/master/data/lbfmodel.yaml"

frontalize_model_name = "model_dlib"
frontalize_model_path = "model_checkpoints/model3Ddlib.mat"

eye_mask_mat = "eyemask"
eye_mask_mat_path = "model_checkpoints/eyemask.mat"

In [ ]:
for filename, url in zip([haarcascade, LBFmodel], [haarcascade_url, LBFmodel_url]):
    if os.path.exists(filename):
        print("File exists")
    else:
        urlreq.urlretrieve(url, filename)
        print("File downloaded")

In [ ]:
images = []

for img_name in os.listdir(base_folder):
    img_path = os.path.join(base_folder, img_name)
    img = cv2.imread(img_path)
    if img is not None:
        images.append(img)

In [ ]:
face_detector = cv2.CascadeClassifier(haarcascade)
landmark_detector = cv2.face.createFacemarkLBF()
landmark_detector.loadModel(LBFmodel)

In [ ]:
def normalize_face(img):
    model3D = frontalize.ThreeD_Model(frontalize_model_path, frontalize_model_name)

    gray = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)
    faces = face_detector.detectMultiScale(gray, scaleFactor=1.1, minNeighbors=5)
    if len(faces) == 0:
        raise RuntimeError("No faces detected.")

    main_face = np.array([max(faces, key=lambda rect: rect[2] * rect[3])])
    retval, landmarks = landmark_detector.fit(gray, main_face)
    if not retval or len(landmarks) == 0:
        raise RuntimeError("Could not detect landmarks.")

    # OpenCV returns landmarks as a list, where each element is an array of shape (1, 68, 2).
    lmarks = landmarks[0][0]
    proj_matrix, _, _, _ = calib.estimate_camera(model3D, lmarks)

    eyemask = np.asarray(scio.loadmat(eye_mask_mat_path)[eye_mask_mat])
    frontal_raw, frontal_sym = frontalize.frontalize(
        img, proj_matrix, model3D.ref_U, eyemask
    )

    return frontal_raw, frontal_sym


def obtain_only_face(i, frontal_view):
    faces = face_detector.detectMultiScale(
        frontal_view, scaleFactor=1.1, minNeighbors=5
    )

    if len(faces) == 0:
        raise RuntimeError(f"No faces detected (after frontalization) {i}.")

    main_face = np.array([max(faces, key=lambda rect: rect[2] * rect[3])])
    _, landmarks = landmark_detector.fit(frontal_view, main_face)

    lmarks = landmarks[0][0]
    hull = cv2.convexHull(np.array(lmarks, dtype=np.int32))

    min_x = min(lmarks, key=lambda p: p[0])[0]
    max_x = max(lmarks, key=lambda p: p[0])[0]
    min_y = min(lmarks, key=lambda p: p[1])[1]
    max_y = max(lmarks, key=lambda p: p[1])[1]

    mask = np.zeros((frontal_view.shape[0], frontal_view.shape[1]), dtype=np.uint8)
    cv2.fillPoly(mask, [hull], 255)

    masked_face = frontal_view.copy()
    if masked_face.dtype != np.uint8:
        masked_face = np.uint8(np.clip(masked_face, 0, 255))

    masked_face[mask == 0] = 0
    masked_face = masked_face[
        int(min_y) - 5 : int(max_y) + 5, int(min_x) - 5 : int(max_x) + 5
    ]

    masked_face = cv2.cvtColor(masked_face, cv2.COLOR_BGR2GRAY)
    resized_face = cv2.resize(masked_face, face_image_target_size)
    return resized_face


def normalize_list(image_list, useSym):
    frontalized_input = [normalize_face(x)[1 if useSym else 0] for x in image_list]

    cropped_input = [obtain_only_face(i, x) for i, x in enumerate(image_list)]
    frontalized_cropped_input = [
        obtain_only_face(i, x) for i, x in enumerate(frontalized_input)
    ]

    return cropped_input, frontalized_cropped_input

In [ ]:
cropped_input, frontalized_cropped_input = normalize_list(images, useSym=False)

In [ ]:
num_images = len(images)
fig, axes = plt.subplots(num_images, 3, figsize=(6, 2 * num_images))
for i in range(num_images):
    axes[i, 0].imshow(cv2.cvtColor(images[i], cv2.COLOR_BGR2RGB))
    axes[i, 0].axis("off")
    if i == 0:
        axes[i, 0].set_title(f"Original")

    axes[i, 1].imshow(cv2.cvtColor(cropped_input[i], cv2.COLOR_BGR2RGB))
    axes[i, 1].axis("off")
    if i == 0:
        axes[i, 1].set_title(f"Cropped")

    axes[i, 2].imshow(cv2.cvtColor(frontalized_cropped_input[i], cv2.COLOR_BGR2RGB))
    axes[i, 2].axis("off")
    if i == 0:
        axes[i, 2].set_title(f"Front")

plt.tight_layout()
plt.show()